In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

### Load data

In [2]:
boston_data = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/Boston_Housing/Training_set_boston.csv")
boston_data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,15.02340,0.0,18.10,0.0,0.6140,5.304,97.3,2.1007,24.0,666.0,20.2,349.48,24.91,12.0
1,0.62739,0.0,8.14,0.0,0.5380,5.834,56.5,4.4986,4.0,307.0,21.0,395.62,8.47,19.9
2,0.03466,35.0,6.06,0.0,0.4379,6.031,23.3,6.6407,1.0,304.0,16.9,362.25,7.83,19.4
3,7.05042,0.0,18.10,0.0,0.6140,6.103,85.1,2.0218,24.0,666.0,20.2,2.52,23.29,13.4
4,0.72580,0.0,8.14,0.0,0.5380,5.727,69.5,3.7965,4.0,307.0,21.0,390.95,11.28,18.2


In [3]:
# separate IV and DV (input and output)
X = boston_data.drop('MEDV', axis=1)
y = boston_data.MEDV

### Split the data

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
n_features = X.shape[1]
n_features

13

### Training

1. Define model
2. Compile model
3. Fit the model
4. Make predictions
5. Evaluate the model

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from numpy.random import seed

In [7]:
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(n_features,)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))

In [8]:
from tensorflow.keras.optimizers import RMSprop

lr = 0.01
optimizer = RMSprop(lr)

In [9]:
model.compile(loss='mean_squared_error', optimizer=optimizer)

In [10]:
seed_value = 42
seed(seed_value)

import os
os.environ['PYTHONHASHSEED']=str(seed_value)

import random
random.seed(seed_value)

tf.random.set_seed(seed_value)
model.fit(X_train,y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
11/11 [==============================] - 0s 1ms/step - loss: 1243.1578
Epoch 2/10
11/11 [==============================] - 0s 1ms/step - loss: 114.0591
Epoch 3/10
11/11 [==============================] - 0s 1ms/step - loss: 109.9059
Epoch 4/10
11/11 [==============================] - 0s 1ms/step - loss: 88.1774
Epoch 5/10
11/11 [==============================] - 0s 1000us/step - loss: 70.2196
Epoch 6/10
11/11 [==============================] - 0s 1ms/step - loss: 75.5323
Epoch 7/10
11/11 [==============================] - 0s 1ms/step - loss: 105.7637
Epoch 8/10
11/11 [==============================] - 0s 1000us/step - loss: 79.0329
Epoch 9/10
11/11 [==============================] - 0s 1ms/step - loss: 80.4028
Epoch 10/10
11/11 [==============================] - 0s 1ms/step - loss: 65.6602


In [11]:
model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 2ms/step - loss: 77.7790


77.77896881103516

### Hyperparameter tuning

Most problems require a lot of hyperparameter tuning. Few steps I've learned, experienced and found:
- **Training loss** should decrease, steep then slowly as it the slope reaches zero.
- If **training loss** does not converge, train for more epochs.
- If **training loss** decreases too slowly, increase `learning rate`.
- High `learning rate` may prevent the convergence of loss.
- If **training loss** has high variance, decrease `learning rate`.
- Lowering `learning rate` and increasing `epochs`/`batch size` can help loss convergence.
- Small `batch size` can cause instability, try large size then decrease as degradation begins to show.
- Fit as much training data as possible into memory by adjusting `batch size`.

#### Implementation of hyperparameter tuning with `sklearn`

1. Define the model
2. Define hyperparameters grid to be validated
3. Run `GridSearchCV` process
4. Select the best model.

In [12]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

def buildModel(optimizer=RMSprop(0.1)):
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_features,)))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mse', metrics=['mse'], optimizer=optimizer)
    return model

batch_size = [10, 20, 30, 40, 50]
epochs = [10, 100, 200]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [13]:
K_model = KerasRegressor(build_fn=buildModel, verbose=1)

grid = GridSearchCV(
    estimator=K_model, param_grid=param_grid,
    scoring="neg_mean_squared_error",
    cv=5, n_jobs=-1)

grid_result = grid.fit(X_train, y_train)

Epoch 1/200
33/33 [==============================] - 0s 937us/step - loss: 27003.4868 - mse: 27003.4868
Epoch 2/200
33/33 [==============================] - 0s 875us/step - loss: 186.8436 - mse: 186.8436
Epoch 3/200
33/33 [==============================] - 0s 907us/step - loss: 181.0878 - mse: 181.0878
Epoch 4/200
33/33 [==============================] - 0s 844us/step - loss: 241.1153 - mse: 241.1153
Epoch 5/200
33/33 [==============================] - 0s 844us/step - loss: 90.4568 - mse: 90.4568
Epoch 6/200
33/33 [==============================] - 0s 875us/step - loss: 90.7784 - mse: 90.7784
Epoch 7/200
33/33 [==============================] - 0s 844us/step - loss: 91.5150 - mse: 91.5150
Epoch 8/200
33/33 [==============================] - 0s 844us/step - loss: 93.7407 - mse: 93.7407
Epoch 9/200
33/33 [==============================] - 0s 906us/step - loss: 93.1169 - mse: 93.1169
Epoch 10/200
33/33 [==============================] - 0s 844us/step - loss: 87.7091 - mse: 87.7091
Epoch 1

In [14]:
print(f'Best params: {grid_result.best_params_}')

Best params: {'batch_size': 10, 'epochs': 200}


In [15]:
from sklearn.model_selection import cross_val_score

results = cross_val_score(grid.best_estimator_, X_test, y_test, cv=5)
print(results.mean())
print(results.std())

Epoch 1/200
7/7 [==============================] - 0s 1ms/step - loss: 17779.0952 - mse: 17779.0944
Epoch 2/200
7/7 [==============================] - 0s 1ms/step - loss: 618.7091 - mse: 618.7091
Epoch 3/200
7/7 [==============================] - 0s 2ms/step - loss: 595.5793 - mse: 595.5793
Epoch 4/200
7/7 [==============================] - 0s 1ms/step - loss: 654.1107 - mse: 654.1107
Epoch 5/200
7/7 [==============================] - 0s 2ms/step - loss: 587.6349 - mse: 587.6349
Epoch 6/200
7/7 [==============================] - 0s 1ms/step - loss: 523.1590 - mse: 523.1590
Epoch 7/200
7/7 [==============================] - 0s 1ms/step - loss: 541.6254 - mse: 541.6254
Epoch 8/200
7/7 [==============================] - 0s 1ms/step - loss: 524.6236 - mse: 524.6236
Epoch 9/200
7/7 [==============================] - 0s 1ms/step - loss: 399.7749 - mse: 399.7749
Epoch 10/200
7/7 [==============================] - 0s 1ms/step - loss: 457.5607 - mse: 457.5607
Epoch 11/200
7/7 [=================

#### Implementing hyperparameter tuning with Keras

1. Define model
2. Define hyperparameters
3. Run tuner process
4. Select best model/parameters

In [16]:
import kerastuner as kt

def buildModel(hp):
    model = Sequential(
        [
            Dense(10, activation='relu', input_shape=(n_features,)),
            Dense(8, activation='relu'),
            Dense(1)
        ]
    )
    hp_lr = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])
    optimizer = RMSprop(learning_rate=hp_lr)
    model.compile(loss='mse', metrics=['mse'], optimizer=optimizer)
    return model

In [19]:
tuner_rs = kt.RandomSearch(
    buildModel,
    objective='mse',
    seed=42,
    max_trials=5,
    directory='random_search/test_01'
)

In [20]:
tuner_rs.search(X_train, y_train, epochs=10, validation_split=0.2, verbose=1)

Trial 4 Complete [00h 00m 01s]
mse: 80.45980072021484

Best mse So Far: 80.45980072021484
Total elapsed time: 00h 00m 04s
INFO:tensorflow:Oracle triggered exit


In [21]:
best_model = tuner_rs.get_best_models(num_models=1)
best_model

In [22]:
best_model = best_model[0]
best_model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 2ms/step - loss: 51.9345 - mse: 51.9345


[55.21622848510742, 55.21622848510742]

In [23]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                140       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 237
Trainable params: 237
Non-trainable params: 0
_________________________________________________________________
